# **Extração de dados da Planilha**

**Autor:** Davi Vago Soares  
**Data:** 25-08-2024


In [11]:
#Bibliotecas utilizadas

import pandas as pd
import numpy as np
import pyodbc
import warnings
warnings.filterwarnings('ignore')

In [12]:
#definindo local do arquivo utilizado

caminho = r"C:\Users\davis\OneDrive\Repositórios\Moer\moer\database-archives\registro-extracoes.xlsx"

df = pd.read_excel(caminho)

In [13]:
df.columns

Index(['id_extracao_pk', 'id_moedor_fk', 'Moedor', 'id_metodo_fk', 'Método',
       'id_cafe_fk', 'vl_peso_cafe', 'vl_nota', 'vl_pesoagua',
       'vl_temperatura', 'vl_regulagem_moedor', 'ds_sensorial', 'ds_corpo',
       'dt_dia_extracao', 'ts_tempo_extracao'],
      dtype='object')

In [14]:
#Verificando tipos

df.dtypes

id_extracao_pk                float64
id_moedor_fk                    int64
Moedor                         object
id_metodo_fk                    int64
Método                         object
id_cafe_fk                    float64
vl_peso_cafe                  float64
vl_nota                       float64
vl_pesoagua                   float64
vl_temperatura                float64
vl_regulagem_moedor           float64
ds_sensorial                   object
ds_corpo                       object
dt_dia_extracao        datetime64[ns]
ts_tempo_extracao              object
dtype: object

In [15]:
#verificando dataframe

df.head()

,id_extracao_pk,id_moedor_fk,Moedor,id_metodo_fk,Método,id_cafe_fk,vl_peso_cafe,vl_nota,vl_pesoagua,vl_temperatura,vl_regulagem_moedor,ds_sensorial,ds_corpo,dt_dia_extracao,ts_tempo_extracao
0,1.0,1,Kingrinder K6,3,Hario V60 acrilico,1.0,20.5,3.0,301.0,97.0,100.0,Frutas amarelas,Macio,2024-08-24,00:03:01.0000000
1,9.0,1,Kingrinder K6,3,Hario V60 acrilico,1.0,20.9,4.0,305.2,94.4,100.0,Rapadura,Médio,2024-08-27,00:03:40.0000000
2,10.0,1,Kingrinder K6,2,Aeropress,1.0,17.0,4.0,190.0,98.0,80.0,Chocolate Amargo,Encorpado,2024-08-28,00:00:30.0000000
3,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [16]:
#Conectando ao banco de dados

try:
    server = 'localhost\\SQLEXPRESS'
    database = 'coffee'
    conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
    print("Conexão bem-sucedida!")
except pyodbc.Error as ex:
    sqlstate = ex.args[1]
    print(f"Erro de Conexão: {sqlstate}")

Conexão bem-sucedida!


In [17]:
#buscando maior id no banco para atualização incremental

# Define o comando SQL
sql_comando = """
SELECT MAX(id_extracao_pk) AS max_id
FROM [coffee].[dbo].[fato_extracoes]
"""

# Executa a consulta e cria um DataFrame
maiorId = pd.read_sql_query(sql_comando, conn)
maiorId

# Obtém o valor de max_id
valor_max_id = maiorId['max_id'].iloc[0]
valor_max_id


9

In [18]:
# Filtra o DataFrame df onde id_extracao_pk é maior que o valor de max_id
df_filtrado = df[df['id_extracao_pk'] > valor_max_id]

In [19]:
#Selecionando colunas

df = df_filtrado[['id_moedor_fk', 'id_metodo_fk',
       'id_cafe_fk', 'vl_peso_cafe', 'vl_nota', 'vl_pesoagua',
       'vl_temperatura', 'vl_regulagem_moedor', 'ds_sensorial', 'ds_corpo',
       'dt_dia_extracao', 'ts_tempo_extracao']].copy()

df

,id_moedor_fk,id_metodo_fk,id_cafe_fk,vl_peso_cafe,vl_nota,vl_pesoagua,vl_temperatura,vl_regulagem_moedor,ds_sensorial,ds_corpo,dt_dia_extracao,ts_tempo_extracao
2,1,2,1.0,17.0,4.0,190.0,98.0,80.0,Chocolate Amargo,Encorpado,2024-08-28,00:00:30.0000000


In [20]:
# Escrevendo dados no banco

cursor = conn.cursor()

sql_comando = """INSERT INTO [coffee].[dbo].[fato_extracoes] (
       [id_moedor_fk]
      ,[id_metodo_fk]
      ,[id_cafe_fk]
      ,[vl_peso_cafe]
      ,[vl_nota]
      ,[vl_pesoagua]
      ,[vl_temperatura]
      ,[vl_regulagem_moedor]
      ,[ds_sensorial]
      ,[ds_corpo]
      ,[dt_dia_extracao]
      ,[ts_tempo_extracao]) 
      VALUES ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""

# Iterando pelas linhas do DataFrame
for array in df.values:
    # Executa o comando SQL substituindo os placeholders pelos valores da linha
    cursor.execute(sql_comando, tuple(array))

# Confirma as mudanças no banco de dados
conn.commit()